# 수행한 것
- 제목만 전처리 

## 데이터로드

In [42]:
import pandas as pd
import numpy as np
import seaborn as sns

# plotting 환경 설정
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
plt.rcParams['figure.figsize'] = (5,5) # 그래프 크기
plt.rcParams['font.family'] = 'Malgun Gothic' # 글꼴
plt.rcParams['font.size'] = 12 # 글꼴 크기
plt.rcParams['axes.unicode_minus'] = False # 마이너스 표기

# 그래프가 노트북 안에 보이게 하기 위해
%matplotlib inline

In [43]:
# 데이터를 구분하는 문자를 지정하는 옵션
con8 = pd.read_csv('../data/데이터스쿨3차_2308월/데이터스쿨_3차_콘텐츠_2308.csv', encoding = 'cp949', sep = '\t')
con9 = pd.read_csv('../data/데이터스쿨3차_2309월/데이터스쿨_3차_콘텐츠_2309.csv', encoding = 'cp949', sep = '\t')

In [44]:
c8 = con8.copy()
c9 = con9.copy()

## content 전처리

In [45]:
# con 데이터프레임을 header가 같은 열을 기준으로 합치기
con89 = pd.concat([con8, con9], ignore_index=True)
con89.head()

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt
0,65968000,(영어더빙)핑크퐁 동물 동요,핑크퐁 동물 동요 20회,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,0:01,20230813071155
1,65968000,(영어더빙)핑크퐁 동물 동요,핑크퐁 동물 동요 20회,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,0:01,20230813071215
2,66056000,라바 키즈동요(한국어) Part4,라바 키즈동요(한국어) Part4 17회,키즈,기타,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...,"엘로우,레드",0:02,20230808085440
3,66056000,라바 키즈동요(한국어) Part4,라바 키즈동요(한국어) Part4 17회,키즈,기타,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...,"엘로우,레드",0:02,20230808085432
4,63086000,지니강이 플러스 시즌7.,지니강이 플러스 시즌7. 26회,키즈,기타,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,"헤이지니,럭키강이",0:11,20230827145542


In [46]:
from pandas._libs.tslibs.offsets import Hour
import re
def preprocessing_con(df):
  # 특수문자 정리

  # series_nm 괄호와 그 안의 내용 제거
  df['series_nm'] = df['series_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'…', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace('-', ' ')
  df['series_nm'] = df['series_nm'].str.rstrip('.')

  # super_asset_nm 변경
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'…', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace('-', ' ')
  df['super_asset_nm'] = df['super_asset_nm'].str.rstrip('.')

  # 회, 회. 제거
  df['series_nm'] = df['series_nm'].str.replace(r'무삭제판|무삭제', '', regex = True)
  df['series_nm'] = df['series_nm'].str.replace(r'실사판', '', regex = True)
  df['series_nm'] = df['series_nm'].str.replace(r'\d+회$', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'\d+회\.', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'\d+화$', '', regex = True)
  df['series_nm'] = df['series_nm'].str.strip()
  df['series_nm'] = df['series_nm'].str.rstrip('.')

  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'무삭제판|무삭제', '', regex = True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'실사판', '', regex = True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\d+회$', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\d+회\.', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\d+화$', '', regex = True)
  df['super_asset_nm'] = df['super_asset_nm'].str.strip()
  df['super_asset_nm'] = df['super_asset_nm'].str.rstrip('.')

  # 'series_nm'이 null인 경우, 'super_asset_nm'의 값으로 대체
  df['series_nm'].fillna(df['super_asset_nm'], inplace=True)

  # disp_rtm 문자열을 분 단위로 변경
  def convert_runtime(runtime_str):
    # 입력값이 NaN이면 0 반환
    if pd.isna(runtime_str):
        return 0

    hours = int(runtime_str.split(':')[0])
    minutes = int(runtime_str.split(':')[1])
    total_minutes = hours * 60 + minutes
    return total_minutes

  df['disp_rtm'] = df['disp_rtm'].apply(convert_runtime)

  # 셋탑번호가 66056000인 것 삭제 - content 에서 이상치 아이디로 판별
  df = df[df['subsr'] != 66056000]

  # index reset
  df = df.reset_index(drop=True)

  return df

In [47]:
df = preprocessing_con(con89)
df.head()

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt
0,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071155
1,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071215
2,63086000,지니강이 플러스 시즌7,지니강이 플러스 시즌7,키즈,기타,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,"헤이지니,럭키강이",11,20230827145542
3,63086000,지니강이 플러스 시즌9,지니강이 플러스 시즌9,키즈,기타,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,"헤이지니,럭키강이",11,20230827152417
4,62733000,엘리가간다 시즌12,엘리가간다 시즌12,키즈,기타,미니 시리얼 팬케이크 만들기.,-,13,20230816112816


In [48]:
# Category 구분
df['Category'] = np.where(df['ct_cl'] == '영화', '영화', np.where(df['ct_cl'] == '키즈', '키즈', 'TV프로그램'))
df

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt,Category
0,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071155,키즈
1,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071215,키즈
2,63086000,지니강이 플러스 시즌7,지니강이 플러스 시즌7,키즈,기타,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,"헤이지니,럭키강이",11,20230827145542,키즈
3,63086000,지니강이 플러스 시즌9,지니강이 플러스 시즌9,키즈,기타,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,"헤이지니,럭키강이",11,20230827152417,키즈
4,62733000,엘리가간다 시즌12,엘리가간다 시즌12,키즈,기타,미니 시리얼 팬케이크 만들기.,-,13,20230816112816,키즈
...,...,...,...,...,...,...,...,...,...,...
21160,64551000,슈퍼 토끼,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,12,20230930121656,키즈
21161,64551000,슈퍼 토끼,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,12,20230930121646,키즈
21162,64551000,슈퍼 토끼,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,12,20230930121655,키즈
21163,64551000,슈퍼 토끼,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,12,20230930121655,키즈


In [49]:
# name에서 띄어쓰기 전부 제거한 열 생성
df['rename'] = df['series_nm'].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
df

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt,Category,rename
0,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071155,키즈,핑크퐁동물동요
1,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071215,키즈,핑크퐁동물동요
2,63086000,지니강이 플러스 시즌7,지니강이 플러스 시즌7,키즈,기타,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,"헤이지니,럭키강이",11,20230827145542,키즈,지니강이플러스시즌7
3,63086000,지니강이 플러스 시즌9,지니강이 플러스 시즌9,키즈,기타,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,"헤이지니,럭키강이",11,20230827152417,키즈,지니강이플러스시즌9
4,62733000,엘리가간다 시즌12,엘리가간다 시즌12,키즈,기타,미니 시리얼 팬케이크 만들기.,-,13,20230816112816,키즈,엘리가간다시즌12
...,...,...,...,...,...,...,...,...,...,...,...
21160,64551000,슈퍼 토끼,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,12,20230930121656,키즈,슈퍼토끼
21161,64551000,슈퍼 토끼,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,12,20230930121646,키즈,슈퍼토끼
21162,64551000,슈퍼 토끼,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,12,20230930121655,키즈,슈퍼토끼
21163,64551000,슈퍼 토끼,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,12,20230930121655,키즈,슈퍼토끼


In [50]:
# vod_list에서 vod_id 가져오기
vod_list = pd.read_csv('../data/전처리data/vod_all.csv', encoding='cp949')
vod_list

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,rename,counts,vod_id
0,와이낫크루 뜻밖의 여행,NaN,"KCM,권은비,조정식,박현규",NaN,TV 연예/오락,기타,TV프로그램,69분,"# 오늘, 이 동네는 우리가 알잘딱깔센 접수한다! 뻔한 랜드마크는 궁금하지도 않고,...",와이낫크루뜻밖의여행,16,0
1,그레이트 뷰티,NaN,"토니 세르빌로,사브리나 페릴리,세레나 그랜디,이사벨라 페라리",NaN,영화,기타,영화,0,(HD)그레이트 뷰티,그레이트뷰티,1,1
2,해안선,NaN,"장동건,김정학,박지아,유해진",NaN,영화,액션/어드벤쳐,영화,0,(SD)해안선,해안선,1,2
3,아는 형님,NaN,"강호동,서장훈,김영철,이수근,김희철,민경훈,이상민",NaN,TV 연예/오락,기타,TV프로그램,87분,"(구)동구 윤시윤, (구)호동 이호원 등장! 아형군 탄생리 이장 선발전! 후보 1번...",아는형님,44,3
4,베놈,NaN,"톰 하디,미셀 윌리엄스,리즈 아메드,제니 슬레이트,레이드 스콧,스콧 헤이즈,우디 해...",NaN,영화,액션/어드벤쳐,영화,140분,(본편+부가영상) 서비스 해지 시까지 시청 가능. 좋은 놈 에디브룩(톰 하디) vs...,베놈,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...
4710,베놈 2: 렛 데어 비 카니지,NaN,"톰 하디,미셀 윌리엄스,나오미 해리스,레이드 스콧,스티븐 그레이엄,우디 해럴슨",NaN,영화,액션/어드벤쳐,영화,2분,히어로의 시대는 끝났다. 미워할 수 없는 빌런 히어로 베놈과 완벽한 파트너가 된 에...,베놈2:렛데어비카니지,11,4710
4711,성월동화,NaN,"장국영,토키와 타카코,양자경,이찬삼,고첩,혜천사,요계지,황가락",NaN,영화,멜로,영화,0,히토미는 타츠야와 결혼을 앞두고 홍콩에서의 생활을 위해 광동어를 배우고 있다. 어느...,성월동화,3,4711
4712,다큐인사이트 코드블루,NaN,유승호,NaN,TV 시사/교양,기타,TV프로그램,47분,히포크라테스 선서를 외치고 의사들이 출발선에 서는 3월. 전남대병원에 발 디딘 새내...,다큐인사이트코드블루,1,4712
4713,캠프 제로베이스원,NaN,제로베이스원,NaN,TV 연예/오락,기타,TV프로그램,109분,힐링 끝! 미션 시작! 마지막까지 호락호락하지 않은 제주도에서의 로즈 미션!,캠프제로베이스원,1,4713


In [51]:
# rename, Category 기준으로 vod_id 붙임
df_add_id = pd.merge(df, vod_list[['rename', 'Category', 'vod_id']], on=['rename', 'Category'], how='left')
df_add_id

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt,Category,rename,vod_id
0,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071155,키즈,핑크퐁동물동요,882.0
1,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071215,키즈,핑크퐁동물동요,882.0
2,63086000,지니강이 플러스 시즌7,지니강이 플러스 시즌7,키즈,기타,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,"헤이지니,럭키강이",11,20230827145542,키즈,지니강이플러스시즌7,1622.0
3,63086000,지니강이 플러스 시즌9,지니강이 플러스 시즌9,키즈,기타,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,"헤이지니,럭키강이",11,20230827152417,키즈,지니강이플러스시즌9,2043.0
4,62733000,엘리가간다 시즌12,엘리가간다 시즌12,키즈,기타,미니 시리얼 팬케이크 만들기.,-,13,20230816112816,키즈,엘리가간다시즌12,1958.0
...,...,...,...,...,...,...,...,...,...,...,...,...
21160,64551000,슈퍼 토끼,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,12,20230930121656,키즈,슈퍼토끼,544.0
21161,64551000,슈퍼 토끼,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,12,20230930121646,키즈,슈퍼토끼,544.0
21162,64551000,슈퍼 토끼,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,12,20230930121655,키즈,슈퍼토끼,544.0
21163,64551000,슈퍼 토끼,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,12,20230930121655,키즈,슈퍼토끼,544.0


In [52]:
df_add_id[df_add_id['vod_id'].isna()]

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt,Category,rename,vod_id
4974,66526000,엘리멘탈,엘리멘탈,TV애니메이션,기타,"불, 물, 땅, 공기의 네 원소들이 함께 사는 엘리멘트 시티를 배경으로 펼쳐지는 영...","레아 루이스,마무두 애시,웬디 맥렌던 커비,메이슨 베르트하이머",2,20230814053117,TV프로그램,엘리멘탈,NaN
4975,67044000,엘리멘탈,엘리멘탈,TV애니메이션,기타,"불, 물, 땅, 공기의 네 원소들이 함께 사는 엘리멘트 시티를 배경으로 펼쳐지는 영...","레아 루이스,마무두 애시,웬디 맥렌던 커비,메이슨 베르트하이머",2,20230814045111,TV프로그램,엘리멘탈,NaN
4976,65367000,엘리멘탈,엘리멘탈,TV애니메이션,기타,"불, 물, 땅, 공기의 네 원소들이 함께 사는 엘리멘트 시티를 배경으로 펼쳐지는 영...","레아 루이스,마무두 애시,웬디 맥렌던 커비,메이슨 베르트하이머",2,20230814180514,TV프로그램,엘리멘탈,NaN
4977,66444000,엘리멘탈,엘리멘탈,TV애니메이션,기타,"불, 물, 땅, 공기의 네 원소들이 함께 사는 엘리멘트 시티를 배경으로 펼쳐지는 영...","레아 루이스,마무두 애시,웬디 맥렌던 커비,메이슨 베르트하이머",2,20230814084446,TV프로그램,엘리멘탈,NaN
4978,66444000,엘리멘탈,엘리멘탈,TV애니메이션,기타,"불, 물, 땅, 공기의 네 원소들이 함께 사는 엘리멘트 시티를 배경으로 펼쳐지는 영...","레아 루이스,마무두 애시,웬디 맥렌던 커비,메이슨 베르트하이머",2,20230814084212,TV프로그램,엘리멘탈,NaN
4979,65442000,엘리멘탈,엘리멘탈,TV애니메이션,기타,"불, 물, 땅, 공기의 네 원소들이 함께 사는 엘리멘트 시티를 배경으로 펼쳐지는 영...","레아 루이스,마무두 애시,웬디 맥렌던 커비,메이슨 베르트하이머",2,20230811000255,TV프로그램,엘리멘탈,NaN
4980,65254000,엘리멘탈,엘리멘탈,TV애니메이션,기타,"불, 물, 땅, 공기의 네 원소들이 함께 사는 엘리멘트 시티를 배경으로 펼쳐지는 영...","레아 루이스,마무두 애시,웬디 맥렌던 커비,메이슨 베르트하이머",2,20230813122703,TV프로그램,엘리멘탈,NaN
4981,65968000,엘리멘탈,엘리멘탈,TV애니메이션,기타,"불, 물, 땅, 공기의 네 원소들이 함께 사는 엘리멘트 시티를 배경으로 펼쳐지는 영...","레아 루이스,마무두 애시,웬디 맥렌던 커비,메이슨 베르트하이머",2,20230813081721,TV프로그램,엘리멘탈,NaN
4982,61524000,엘리멘탈,엘리멘탈,TV애니메이션,기타,"불, 물, 땅, 공기의 네 원소들이 함께 사는 엘리멘트 시티를 배경으로 펼쳐지는 영...","레아 루이스,마무두 애시,웬디 맥렌던 커비,메이슨 베르트하이머",2,20230813081932,TV프로그램,엘리멘탈,NaN
4983,65254000,엘리멘탈,엘리멘탈,TV애니메이션,기타,"불, 물, 땅, 공기의 네 원소들이 함께 사는 엘리멘트 시티를 배경으로 펼쳐지는 영...","레아 루이스,마무두 애시,웬디 맥렌던 커비,메이슨 베르트하이머",2,20230813122935,TV프로그램,엘리멘탈,NaN


In [53]:
# 제목만 전처리
df_add_id.to_csv('con_prepro_series_nm.csv', index=False, encoding="cp949")